In [ ]:
!pip install pydicom

In [ ]:
import torch, os, time, copy
import numpy as np
from copy import deepcopy
import pydicom
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

colour_mapping = {
    'TP' : (1, 0, 0)   ,     # red
    'FP' : (1, 1, 0) ,     # yellow
    'FN' : (0, 0, 1),        # blue
    'sky': (0, 1, 0)   ,     # green
    }

def apply_mask(vol, mask, hu=(-100, 400), colour_mapping=None):
    # input numpy array
    if colour_mapping is None:
        colour_mapping = {
            'TP' : (1, 0, 0)   ,     # red
            'FP' : (1, 1, 0) ,     # yellow
            'FN' : (0, 0, 1),        # blue
            'sky': (0, 1, 0)   ,     # green
            }
    _vol = vol.copy()
    _mask = mask.copy()
    # preprocess
    _vol = np.clip(_vol, hu[0], hu[1])
    _vol = (_vol - _vol.min()) / (_vol.max() - _vol.min())
    _mask = np.clip(_mask, 0, 1)
    # add channel RGB
    _vol = np.stack((_vol, )*3, axis=-1)
    # apply mask
    _vol[_mask==1] = colour_mapping['TP']

    return _vol 

In [28]:
DIR = '/content/drive/My Drive/Tumor_src/Datasets/3Dircad/Origin/3Dircadb1.15'
vol_path = DIR + '/PATIENT_DICOM'
liver_path = DIR + '/MASKS_DICOM/liver'

In [ ]:
for i in range(125):
  vol = pydicom.dcmread(vol_path + '/image_' + str(i)).pixel_array
  liver = pydicom.dcmread(liver_path + '/image_' + str(i)).pixel_array
  vol_n = apply_mask(vol, liver)

  _, ax = plt.subplots()
  ax.imshow(vol_n)
  ax.axis('off')
  plt.tight_layout()
  plt.savefig(f'/content/drive/My Drive/Tumor_src/mask_png/image_{i}.png')
  